In [1]:
import findspark
findspark.init("/opt/spark")

from pyspark.sql import SparkSession 

spark = (
    SparkSession
        .builder
        .master('local[4]') 
        .appName('mi_app')
        .getOrCreate()
)

24/09/05 19:04:46 WARN Utils: Your hostname, zenlo resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
24/09/05 19:04:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/05 19:04:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/05 19:04:47 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: 8f:01:5a:5f:c8:2c:4c:44
24/09/05 19:04:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/05 19:04:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = spark.range(5).toDF('num')
df.show()

+---+
|num|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [5]:
# 1. defino funcion
def power3(double_val):
    return double_val**3

power3(4)

64

In [22]:
# 2. registro function
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType

# esto registra la función, como dataframe function. Si se quiere ejecutar como SQL function dara error.
power3udf = F.udf(power3)

# esto registra como SQL function. Como la función devuelve int, el tipo causa que la devolucion sea NULL
spark.udf.register('power3udf_beta', power3, DoubleType())
# en este caso es el tipo correcto
spark.udf.register('power3udf_gamma', power3, IntegerType())

24/09/05 19:39:04 WARN SimpleFunctionRegistry: The function power3udf_beta replaced a previously registered function.


<function __main__.power3(double_val)>

In [25]:
# 3. utilizo
(
    df
    .withColumn('new', power3udf(F.col('num')))
    .selectExpr("num", "new", "power3udf_beta(num)", "power3udf_gamma(num)")
    .show(5)
)

+---+---+-------------------+--------------------+
|num|new|power3udf_beta(num)|power3udf_gamma(num)|
+---+---+-------------------+--------------------+
|  0|  0|               NULL|                   0|
|  1|  1|               NULL|                   1|
|  2|  8|               NULL|                   8|
|  3| 27|               NULL|                  27|
|  4| 64|               NULL|                  64|
+---+---+-------------------+--------------------+

